# 라이브러리

In [81]:
options(warn =-1, repr.plot.width=15, repr.plot.height=9)
# ## 패키지 설치
library(dplyr)
library(stringr)
library(ggplot2)
library(gridExtra)
library(scales)
library(randomForest)
library(lubridate)
library(caret)
library(e1071)
library(nnet)
library(class)


# 전처리

In [82]:
### .csv 불러오기
data <- read.csv("./data/refined_data_filtered_major5_subset.csv")

In [83]:
data$근무지역분류 <- as.character(data$근무지역분류)
data$근무지역분류 <- factor(data$근무지역분류)
data$장애유형 <- as.character(data$장애유형)
data$장애유형 <- factor(data$장애유형)
data$중증여부 <- as.character(data$중증여부)
data$중증여부 <- factor(data$중증여부)
data$근무지역 <- as.character(data$근무지역)
data$근무지역 <- factor(data$근무지역)
data$취업직종대분류 <- as.character(data$취업직종대분류)
data$취업직종대분류 <- factor(data$취업직종대분류)
data$연령대 <- as.character(data$연령대)
data$연령대 <- factor(data$연령대)
data$장애유형_중증여부 <- as.character(data$장애유형_중증여부)
data$장애유형_중증여부 <- factor(data$장애유형_중증여부)
data$월 <- as.character(data$월)
data$월 <- factor(data$월)

In [84]:
data$target<-(ifelse(data$취업직종대분류=="경영·행정·사무직","경영·행정·사무직",data$취업직종대분류))

data$target <- ifelse(data$취업직종대분류 %in% c("청소 및 기타 개인서비스직", "음식 서비스직", "미용·예식 서비스직", "영업·판매직", "스포츠·레크리에이션직","경호·경비직"), "서비스직", data$target)

data$target <- ifelse(grepl("^\\d+$", data$target), "기타", data$target)

In [85]:
table(data$target)


경영·행정·사무직             기타         서비스직 
            8961             7474             8544 

In [86]:
# 불필요한 컬럼 제거

data_filtered <- subset(data, select = -c(취업일자,연령,장애유형,중증여부,근무지역))
sum(is.na(data_filtered))

[1] 0

In [87]:
# 연령대
data_filtered$age_20 <- data_filtered$연령대 == 1
data_filtered$age_30 <- data_filtered$연령대 == 2
data_filtered$age_40 <- data_filtered$연령대 == 3
data_filtered$age_50 <- data_filtered$연령대 == 4
data_filtered$age_60 <- data_filtered$연령대 == 5
data_filtered$age_70 <- data_filtered$연령대 == 6

In [88]:
# 장애유형_중증여부 바꾸기
data_filtered$시각장애_중증 <- data_filtered$장애유형_중증여부 == "시각장애-중증"
data_filtered$시각장애_경증 <- data_filtered$장애유형_중증여부 == "시각장애-경증"

data_filtered$지체장애_중증 <- data_filtered$장애유형_중증여부 == "지체장애-중증"
data_filtered$지체장애_경증 <- data_filtered$장애유형_중증여부 == "지체장애-경증"

data_filtered$지적장애_경증 <- data_filtered$장애유형_중증여부 == "지적장애-경증"
data_filtered$지적장애_중증 <- data_filtered$장애유형_중증여부 == "지적장애-중증"

data_filtered$청각장애_경증 <- data_filtered$장애유형_중증여부 == "청각장애-경증"
data_filtered$청각장애_중증 <- data_filtered$장애유형_중증여부 == "청각장애-중증"

data_filtered$정신장애_경증 <- data_filtered$장애유형_중증여부 == "정신장애-경증"
data_filtered$정신장애_중증 <- data_filtered$장애유형_중증여부 == "정신장애-중증"

In [89]:
#   근무지역 분류 바꾸기
data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

data_filtered$강원 <- data_filtered$근무지역분류 == "강원"

data_filtered$전남 <- data_filtered$근무지역분류 == "전남"

data_filtered$충북 <- data_filtered$근무지역분류 == "충북"

data_filtered$경기 <- data_filtered$근무지역분류 == "경기"

data_filtered$인천 <- data_filtered$근무지역분류 == "인천"

data_filtered$서울 <- data_filtered$근무지역분류 == "서울"

data_filtered$울산 <- data_filtered$근무지역분류 == "울산"

data_filtered$대전 <- data_filtered$근무지역분류 == "대전"

data_filtered$부산 <- data_filtered$근무지역분류 == "부산"

data_filtered$전북 <- data_filtered$근무지역분류 == "전북"

data_filtered$광주 <- data_filtered$근무지역분류 == "광주"

data_filtered$충남 <- data_filtered$근무지역분류 == "충남"

data_filtered$대구 <- data_filtered$근무지역분류 == "대구"

data_filtered$경남 <- data_filtered$근무지역분류 == "경남"

data_filtered$제주 <- data_filtered$근무지역분류 == "제주"

data_filtered$세종 <- data_filtered$근무지역분류 == "세종"

data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

In [90]:
# 월
data_filtered$Jan <- data_filtered$월  == 01
data_filtered$Feb <- data_filtered$월  == 02
data_filtered$Mar <- data_filtered$월  == 03
data_filtered$Apr <- data_filtered$월  == 04
data_filtered$May <- data_filtered$월  == 05
data_filtered$Jun <- data_filtered$월  == 06
data_filtered$Jul <- data_filtered$월  == 07
data_filtered$Aug <- data_filtered$월  == 08
data_filtered$Sep <- data_filtered$월  == 09
data_filtered$Oct <- data_filtered$월  == 10
data_filtered$Nov <- data_filtered$월  == 11
data_filtered$Dec <- data_filtered$월  == 12

In [91]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	24979 obs. of  51 variables:
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 2 2 2 2 2 2 2 9 9 9 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 5 5 4 5 4 5 5 3 4 5 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 2 2 1 2 1 2 2 2 8 2 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 1 1 1 1 1 1 5 6 10 1 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 4 4 4 4 4 4 4 4 1 1 ...
 $ target           : chr  "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" ...
 $ age_20           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50           : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ age_60           : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ 시각장애_경증    : logi  FALSE FALSE 

[1] 0

In [92]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	24979 obs. of  51 variables:
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 2 2 2 2 2 2 2 9 9 9 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 5 5 4 5 4 5 5 3 4 5 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 2 2 1 2 1 2 2 2 8 2 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 1 1 1 1 1 1 5 6 10 1 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 4 4 4 4 4 4 4 4 1 1 ...
 $ target           : chr  "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" ...
 $ age_20           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50           : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ age_60           : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ 시각장애_경증    : logi  FALSE FALSE 

[1] 0

In [93]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [94]:
str(data_filtered_subset)
sum(is.na(data_filtered_subset))

'data.frame':	24979 obs. of  47 variables:
 $ 취업직종대분류: Factor w/ 21 levels "건설·채굴직",..: 2 2 2 2 2 2 2 9 9 9 ...
 $ target        : chr  "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" ...
 $ age_20        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50        : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ age_60        : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ 시각장애_경증 : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ 지체장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증 : logi  FA

[1] 0

In [95]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [96]:
str(data_filtered_subset)

'data.frame':	24979 obs. of  47 variables:
 $ 취업직종대분류: Factor w/ 21 levels "건설·채굴직",..: 2 2 2 2 2 2 2 9 9 9 ...
 $ target        : chr  "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" ...
 $ age_20        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50        : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ age_60        : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ 시각장애_경증 : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ 지체장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증 : logi  FA

In [121]:
summary(data_filtered_subset)

              target       age_20          age_30          age_40       
 경영·행정·사무직:8961   Mode :logical   Mode :logical   Mode :logical  
 기타            :7474   FALSE:16671     FALSE:20805     FALSE:21163    
 서비스직        :8544   TRUE :8308      TRUE :4174      TRUE :3816     
   age_50          age_60          age_70        시각장애_중증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:21011     FALSE:21362     FALSE:23883     FALSE:23470    
 TRUE :3968      TRUE :3617      TRUE :1096      TRUE :1509     
 시각장애_경증   지체장애_중증   지체장애_경증   지적장애_경증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:23504     FALSE:22277     FALSE:19194     FALSE:24954    
 TRUE :1475      TRUE :2702      TRUE :5785      TRUE :25       
 지적장애_중증   청각장애_경증   청각장애_중증   정신장애_경증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:15810     FALSE:23879     FALSE:23487     FALSE:24969    
 TRUE :9169      TRUE :1100      TRUE :1492      TRUE :10   

# 변동사항

In [97]:
data_filtered_subset$target <- factor(data_filtered_subset$target)

In [98]:
data_filtered_subset <- subset(data_filtered_subset, select = -c(취업직종대분류))

In [99]:
str(data_filtered_subset)

'data.frame':	24979 obs. of  46 variables:
 $ target       : Factor w/ 3 levels "경영·행정·사무직",..: 1 1 1 1 1 1 1 2 2 2 ...
 $ age_20       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50       : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ age_60       : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ age_70       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증: logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ 시각장애_경증: logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ 지체장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 정신장애_경증: logi  FALSE FALSE FALSE FALSE FA

In [100]:
lm(target~.,data_filtered_subset)


Call:
lm(formula = target ~ ., data = data_filtered_subset)

Coefficients:
      (Intercept)         age_20TRUE         age_30TRUE         age_40TRUE  
         2.648246          -0.981170          -0.940464          -0.774880  
       age_50TRUE         age_60TRUE         age_70TRUE  시각장애_중증TRUE  
        -0.505031          -0.200868                 NA          -0.124094  
시각장애_경증TRUE  지체장애_중증TRUE  지체장애_경증TRUE  지적장애_경증TRUE  
        -0.239131          -0.318188          -0.254062           0.305099  
지적장애_중증TRUE  청각장애_경증TRUE  청각장애_중증TRUE  정신장애_경증TRUE  
         0.308480          -0.095277          -0.052600          -0.370125  
정신장애_중증TRUE           경북TRUE           강원TRUE           전남TRUE  
               NA           0.363080           0.003696           0.324538  
         충북TRUE           경기TRUE           인천TRUE           서울TRUE  
         0.309002           0.243598           0.207244           0.156052  
         울산TRUE           대전TRUE           부산TRUE           전북TRUE  
     

# 시험준비

In [101]:
samp <- createDataPartition(data_filtered_subset$target, p=0.7, list=F)

#   학습용
data.tr <- data_filtered_subset[samp,]

#   검증용
data.test <- data_filtered_subset[-samp,]

In [102]:
#   KNN을 위한 Training Data 준비
data.training <- subset(data.tr, select = -target)
data.training.label <- data.tr$target

In [103]:
#   문제 및 정답 만들기
x   <- subset(data.test, select = -target)
y   <- data.test$target


# 학습시키기

In [104]:
#  랜덤포레스트 학습 시키기
rf <- randomForest(target ~., data.tr, ntree=500)

In [105]:
# SVM 학습시키기
modelsvm <- svm(target~.,data.tr)

In [106]:
# KNN 학습시키기 3,5,7,
# knnmodel <- knn(train=data.training, test=x,cl=data.training.label,k=3)

In [107]:
# nnet 학습시키기
modelnnet <- nnet(target~.,data.tr,size =20)

# weights:  983
initial  value 27143.475597 
iter  10 value 16503.894412
iter  20 value 15765.717602
iter  30 value 15408.708312
iter  40 value 15226.990015
iter  50 value 14996.102860
iter  60 value 14789.753267
iter  70 value 14608.750343
iter  80 value 14486.048473
iter  90 value 14380.720596
iter 100 value 14295.316267
final  value 14295.316267 
stopped after 100 iterations


# 검증하기

In [108]:
#  Random Forest 검증하기
predrf <- predict(rf, x)

In [109]:
#  SVM 검증하기
predmodelsvm <- predict(modelsvm, x)

In [110]:
# KNN 검증하기
# mean(knnmodel==y) 

In [111]:
# nnet 검증하기
prednnet = predict(modelnnet, x,type = "class")    # class : 객관식, raw : 주관식

# 예측력 확인

In [112]:
#  RF 예측력
mean(predrf == y)

[1] 0.5976244

In [113]:
#  SVM 예측력
mean(predmodelsvm == y)

[1] 0.5559856

In [114]:
#  KNN 예측력
# mean(knnmodel == y)

In [115]:
#  nnet 예측력
mean(prednnet == y)

[1] 0.5853463

# 오분류표

In [116]:
#  Random Forest 오분류표
table(predrf,y)

                  y
predrf             경영·행정·사무직 기타 서비스직
  경영·행정·사무직             1890  466      664
  기타                          335 1172      483
  서비스직                      463  604     1416

In [117]:
#  SVM 오분류표
table(predmodelsvm,y)

                  y
predmodelsvm       경영·행정·사무직 기타 서비스직
  경영·행정·사무직             1814  487      792
  기타                          443 1170      589
  서비스직                      431  585     1182

In [118]:
#  KNN 오분류표
# table(y, knnmodel)

In [119]:
# nnet 오분류표
table(prednnet,y)

                  y
prednnet           경영·행정·사무직 기타 서비스직
  경영·행정·사무직             1830  452      624
  기타                          381 1125      508
  서비스직                      477  665     1431

In [120]:
# 새로운 데이터에 대한 예측
predict(rf, (list(age_20=FALSE,age_30=FALSE,age_40=FALSE,age_50=FALSE,age_60=TRUE,age_70=FALSE,
            시각장애_중증=TRUE,시각장애_경증=FALSE,지체장애_경증=FALSE,지체장애_중증=FALSE,지적장애_경증=FALSE,지적장애_중증=FALSE,
            청각장애_경증=FALSE,청각장애_중증=FALSE,정신장애_경증=FALSE,정신장애_중증=FALSE,
            경북=TRUE,강원=FALSE,전남=FALSE,충북=FALSE,경기=FALSE,인천=FALSE,서울=FALSE,울산=FALSE,대전=FALSE,부산=FALSE,
            전북=FALSE,광주=FALSE,충남=FALSE,대구=FALSE,경남=FALSE,제주=FALSE,세종=FALSE,경북=FALSE,
            Jan=TRUE,Feb=FALSE,Mar=FALSE,Apr=FALSE,May=FALSE,Jun=FALSE,Jul=FALSE,Aug=FALSE,Sep=FALSE,
            Oct=FALSE,Nov=FALSE,Dec=FALSE)), type = "prob")

# pred_prob <- predict(rf, new_data, type = "prob")
# pred_prob <- predict(rf, new_data, type = "prob")

,경영·행정·사무직,기타,서비스직
1,0.02,0.942,0.038
